In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser, XMLOutputParser, BaseOutputParser
from langchain.output_parsers import RetryOutputParser, OutputFixingParser
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator
from typing import List
from pprint import pprint
import json

# base_url = "https://api.deepseek.com/"

# model="deepseek-chat"
# llm = ChatOpenAI(model=model, temperature=0, api_key=api_key, base_url=base_url)

base_url="https://ark.cn-beijing.volces.com/api/v3"

model="doubao-1.5-pro-32k-250115"
llm = ChatOpenAI(model=model, temperature=0, api_key=api_key, base_url=base_url)

# base_url="http://localhost:11434"
# model="deepseek-r1"
# llm = ChatOllama(model=model, temperature=0, base_url=base_url)

/Users/bytedance/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
@tool
def get_weather(location: str) -> str:
    """根据location地名返回当地实时天气."""
    return "天气晴朗22度"
llm_with_tools = llm.bind_tools([get_weather])
chain = llm_with_tools #| StrOutputParser()
response = chain.invoke("北京市当前的天气如何")
pprint(response)

AIMessage(content='用户想了解北京市当前的天气，调用 get_weather 函数获取相关信息。', additional_kwargs={'tool_calls': [{'id': 'call_8qhubr3mrq2ipb1h6kygqrdd', 'function': {'arguments': ' {"location": "北京市"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 62, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'doubao-1-5-pro-32k-250115', 'system_fingerprint': None, 'id': '021759885048762c6f82c6173ae2988dd3f2bc3d220408fcfed4d', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--75ed6272-72cd-4683-a41a-8d9cd6f724e1-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '北京市'}, 'id': 'call_8qhubr3mrq2ipb1h6kygqrdd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_

In [4]:
class Joke(BaseModel):
    setup: str = Field(description="笑话中的铺垫问题，必须以?结尾")  #description中的内容通过get_format_instructions()转换成相应的提示词
    punchline: str = Field(description="笑话中回答铺垫问题的部分，通常以抖包袱方式回单铺垫问题，例如谐音，会错意等")
    
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        print(values)
        setup = values.get("setup")
        if setup and setup.strip()[-1] != "？" and setup.strip()[-1] != "?":
            raise ValueError("Badly formed question!")
        return values

parser = PydanticOutputParser(pydantic_object=Joke)
prompt = PromptTemplate(
    template="回答用户的查询.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(prompt.format(query="给我讲一个笑话"))
chain = prompt | llm | parser
chain.invoke({"query": "给我讲一个笑话"})

回答用户的查询.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"description": "笑话中的铺垫问题，必须以?结尾", "title": "Setup", "type": "string"}, "punchline": {"description": "笑话中回答铺垫问题的部分，通常以抖包袱方式回单铺垫问题，例如谐音，会错意等", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}
```
给我讲一个笑话

{'setup': '为什么许仙给白娘子买了顶帽子，白娘子觉得头很重? ', 'punchline': '因为那是顶鸭舌（压蛇）帽。'}


Joke(setup='为什么许仙给白娘子买了顶帽子，白娘子觉得头很重? ', punchline='因为那是顶鸭舌（压蛇）帽。')

In [5]:
parser = JsonOutputParser()
prompt = PromptTemplate(
    template="回答用户的查询.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(prompt.format(query="给我讲一个笑话"))
chain = prompt | llm | parser
chain.invoke({"query": "给我讲一个笑话"})

回答用户的查询.
Return a JSON object.
给我讲一个笑话



{'joke': '小蚂蚁迷路找不到蚁窝，可着急了，恰好看到它的朋友经过，于是冲过去大喊一声：“哥们儿！你…你都如何回蚁窝（回忆我）？”那朋友一愣，然后反问道：“带…带…带着笑或是很沉默？”'}

In [6]:
parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])
prompt = PromptTemplate(
    template="回答用户的查询.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
actor_query = "Generate the shortened filmography for Tom Hanks."
print(prompt.format(query=actor_query))
chain = prompt | llm | parser
chain.invoke({"query": actor_query})

回答用户的查询.
The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
['movies', 'actor', 'film', 'name', 'genre']
```
Generate the shortened filmography for Tom Hanks.



{'movies': [{'actor': [{'name': 'Tom Hanks'},
    {'film': [{'name': 'Forrest Gump'}, {'genre': 'Drama'}]},
    {'film': [{'name': 'Apollo 13'}, {'genre': 'Drama'}]},
    {'film': [{'name': 'The Green Mile'}, {'genre': 'Drama'}]}]}]}

In [7]:
class Action(BaseModel):
    action: str = Field(description="action to take")  #description中的内容通过get_format_instructions()转换成相应的提示词
    action_input: str = Field(description="input to the action")

parser = PydanticOutputParser(pydantic_object=Action)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_value = prompt.format_prompt(query="今天北京天气如何?")
bad_response = '{"action": "search"}' # 假设得到一个错误回答，不符合parser字段要求
try:
    parser.parse(bad_response)
except OutputParserException as e:
    print(e)
retry_parser = RetryOutputParser.from_llm(parser=parser, llm=llm)
retry_parser.parse_with_prompt(bad_response, prompt_value)

Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


Action(action='query_weather', action_input='今天北京的天气')

In [8]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

actor_query = "Generate the shortened filmography for Tom Hanks."
parser = PydanticOutputParser(pydantic_object=Actor)
misformatted = "{'name': 'Tom Hanks', 'film_names: ['Gorrest Gump']}" # 错误的json格式，使用了单引号，而不是双引号
try:
    parser.parse(misformatted)
except OutputParserException as e:
    print(e)
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
fixing_parser.parse(misformatted)

Invalid json output: {'name': 'Tom Hanks', 'film_names: ['Gorrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


Actor(name='Tom Hanks', film_names=['Forrest Gump'])

In [10]:
class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom boolean parser."""
    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        cleaned_text = text.strip().upper()
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            raise OutputParserException(
                f"BooleanOutputParser expected output value to either be "
                f"{self.true_val} or {self.false_val} (case-insensitive). "
                f"Received {text}."
            )
        return cleaned_text == self.true_val.upper()

    @property
    def __type(self) -> str:
        return "boolean_output_parser"

parser = BooleanOutputParser()
try:
    parser.invoke("ye")
except OutputParserException as e:
    print(e)

BooleanOutputParser expected output value to either be YES or NO (case-insensitive). Received ye.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
